In [109]:
import pennylane as qml
from pennylane import numpy as np
from IPython.display import display, Latex

# from numba import jit

# numpy 
import numpy as np_
import scipy as sp

import matplotlib.pyplot as plt

# 2-qubit
dev = qml.device('default.mixed', wires= 2)

# == Hamiltonian setup == 
coeffs = [1]
obs = [qml.PauliZ(0) @ qml.PauliZ(1)]

hamiltonian = qml.Hamiltonian(coeffs, obs)

In [110]:
# == Global variables ==

#     theta_init, tau_1, tau_2, gamma_1, gamma_2, gamma_3 = paras[0], paras[1], paras[2], paras[3], paras[4], paras[5]
Paras_global = np.zeros(6)
Phi_global = 0

Tau, Gamma_ps = 0, 0
Gamma_Dephase = np_.zeros(3)

# == Relation between kraus_gamma vs tau
# def Dephase_factor(tau):
#     global Gamma_Dephase
#     for i in range(len(tau)):
#         Cal_dephase = 1 - np.exp(-2 * tau[i])
#         Gamma_Dephase[i] = Cal_dephase
def Dephase_factor(tau):
    global Gamma_Dephase
    for i in range(len(tau)):
        Cal_dephase = 1 - np.exp(-2 * tau[i])
        Gamma_Dephase[i] = Cal_dephase
            
def Dephase_factor_Get_tau(gamma):
    global Tau 
    
    Cal_tau = -np.log( np.sqrt(1 - gamma))
    Tau = Cal_tau

$$
\rho_{After-Hadamard} = 

\frac{1}{2} 

\begin{bmatrix}

1 & e^{(i\phi - \tau)} \\
e^{(-i\phi - \tau)} & 1

\end{bmatrix}

=

\frac{1}{2} 

\begin{bmatrix}

1 & e^{i\phi} \sqrt{1 - \gamma} \\
e^{-i\phi} \sqrt{1 - \gamma} & 1

\end{bmatrix}$$

$$

where,
\quad \gamma = 
1 - e^{-2 \tau}
$$


$$ 
e^{-\tau} = \sqrt{1 - \gamma}

In [111]:
#  == Generate circuit ==

# circuit_1
@qml.qnode(dev)
def circuit_1(paras):
    # Call global phi
    global Phi_global
    phi = Phi_global
    
    theta_init, tau_1, tau_2, tau_dephase = paras[0], paras[1], paras[2], paras[3]
    
    # Stage_1: RY for pi/2
    qml.RY(np.pi/2, wires=0)
    qml.RY(np.pi/2, wires=1)
    
    # Stage_2: Entangler    
    qml.ApproxTimeEvolution(hamiltonian, tau_1, 1)
    qml.PhaseDamping(Gamma_Dephase[0], wires = 0)
    qml.PhaseDamping(Gamma_Dephase[0], wires = 1)
    
    qml.RX(theta_init, wires = 0)    
    qml.RX(theta_init, wires = 1)    

    qml.RY(-np.pi/2, wires = 0)    
    qml.RY(-np.pi/2, wires = 1)   

    qml.ApproxTimeEvolution(hamiltonian, tau_2, 1)
    qml.PhaseDamping(Gamma_Dephase[1], wires = 0)
    qml.PhaseDamping(Gamma_Dephase[1], wires = 1)

    qml.RY(np.pi/2, wires = 0)    
    qml.RY(np.pi/2, wires = 1) 
    
    # Stage_3: Accumulater
    qml.ApproxTimeEvolution(hamiltonian, phi, 1)
    qml.PhaseDamping(Gamma_Dephase[2], wires = 0)
    qml.PhaseDamping(Gamma_Dephase[2], wires = 1)
    
    qml.RY(-np.pi/2, wires=0)
    qml.RY(-np.pi/2, wires=1)
    
    return qml.density_matrix(wires=[0, 1])

# circuit_2: Post-selection
Gamma_ps = 0

@qml.qnode(dev)
def Post_selection_Dephase(phi):
    
    # theta_init, tau_1, tau_2, gamma_1, gamma_2, gamma_3 = paras[0], paras[1], paras[2], paras[3], paras[4], paras[5]
    global Paras_global, Phi_global, Gamma_ps
    Phi_global = phi
    
    # Get density matrix from circuit_1
    density_matrix = circuit_1(Paras_global)
    qml.QubitDensityMatrix(density_matrix, wires=[0, 1])
    
    # Kraus operator for 2*2 matrix
    K = np.array([ [np.sqrt(1 - Gamma_ps), 0], [0, 1] ])
    K_H = K.conj().T
    
    Numerator = sp.linalg.kron(K, K) @ density_matrix @ sp.linalg.kron(K_H, K_H)
    Denominator = np.trace(sp.linalg.kron(K, K) @ density_matrix @ sp.linalg.kron(K_H, K_H))
    
    rho_ps = Numerator / Denominator
    
    qml.QubitDensityMatrix(rho_ps, wires=[0, 1])
    
    return qml.density_matrix(wires=[0, 1])    

![Alt text](image.png)

In [112]:
# == Cost_function: using global parameters of Entanglers ==

def Cost_function(paras):
    global Paras_global
    # expect: theta_init, tau_1, tau_2, gamma_1, gamma_2, gamma_3 = paras[0], paras[1], paras[2], paras[3], paras[4], paras[5]
    Paras_global = paras
    
    # Call global para for phi
    global Phi_global
    phi = Phi_global
    phi = np.array([ phi ])

    CFI = qml.qinfo.classical_fisher(Post_selection_Dephase)(phi[0])
    return -CFI

In [113]:
paras_random = np_.array([ 1,1,1,1,1,1 ])

Cost_function(paras_random)

array([[-1.26489851]])

In [114]:
# Sweep_Range = np_.arange(1e-3, np.pi*3 + 1e-3, 0.01)

# Sweep_Range[3]

In [115]:
def SLSQP(Range_in, Step_size, Initial_Guess, Gamma_ps_select, Tau_Dephase):
    Sweep_Range = np_.arange(Range_in[0], Range_in[1], Step_size)
    
    # Define global var 
    global Paras_global, Phi_global, Gamma_ps, Gamma_Dephase
    Gamma_ps = Gamma_ps_select
    
    # Generate Data array: CFI = 2-d, Paras = 3-d
    CFI = np_.zeros((len(Tau_Dephase), len(Sweep_Range)))
    Paras_After_Opt = np_.zeros((len(Tau_Dephase), len(Sweep_Range), len(Paras_global)))
    
    for i in range(len(Tau_Dephase)):
        Dephase_factor(Tau_Dephase[i])
        
        for j in range(len(Sweep_Range)):
        Phi_global = Sweep_Range[j]
        After_SLSQP = sp.optimize.minimize(Cost_function, Initial_Guess, method = 'SLSQP')
        
        CFI[j] = -After_SLSQP.fun
        
        for k in range(len(Paras_global)):
            Paras_After_Opt[j][k] = After_SLSQP.x[k]

        
    
    
    
    
    return CFI, Paras_After_Opt

In [116]:
Sweep_Range_test = np_.array([ 1e-4, np.pi * 2 + 1e-4])
Step_size_test = 1e-2
Initial_guess_test = 0.5,0.5,0.5,0.5,0.5,0.5
Gamma_ps_test = 0.5

Tau_select = 0.000000001, 0.05

SLSQP(Sweep_Range_test, Step_size_test, Initial_guess_test, Gamma_ps_test, Tau_select)

KeyboardInterrupt: 